In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import CountVectorizer
from gensim.corpora import Dictionary
from gensim.models.ldamodel import LdaModel
from gensim.models import CoherenceModel
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from datetime import datetime
import nltk
nltk.download('stopwords')
import pandas as pd
import re
import math

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\aditya.baser\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
df = pd.read_csv('combined_csv_wealthmanagement.csv')
df.head()

,date,username,to,replies,retweets,favorites,text,geo,mentions,hashtags,id,permalink
0,12/31/2019 23:45,HT_Great_Lakes,NaN,0,1,1,Thinking about transitioning your business? Yo...,NaN,NaN,#wealthmanagement #hightowergreatlakes #busine...,1.212160e+18,https://twitter.com/HT_Great_Lakes/status/1212...
1,12/31/2019 23:31,ripsawwealth,NaN,0,2,3,http://RipsawWealth.com #2020WillBeTheYearFor ...,NaN,NaN,#2020WillBeTheYearFor #HappyNewYear #happynewy...,1.212150e+18,https://twitter.com/ripsawwealth/status/121215...
2,12/31/2019 23:00,TedHunter1,NaN,0,0,0,#wealthmanagement #maxous,NaN,NaN,#wealthmanagement #maxous,1.212150e+18,https://twitter.com/TedHunter1/status/12121465...
3,12/31/2019 22:45,RAlexJimenez,NaN,0,0,0,#WealthManagement and private banking trends: ...,NaN,@FinTechMagazine,#WealthManagement #wealthtech #fintech #finserv,1.212140e+18,https://twitter.com/RAlexJimenez/status/121214...
4,12/31/2019 22:43,findadvisor,NaN,0,0,0,Is Your Money Ready for Retirement? #Retiremen...,NaN,NaN,#RetirementPlan #FinancialPlan #WealthManageme...,1.212140e+18,https://twitter.com/findadvisor/status/1212142...


In [3]:
df_tweets = df[["date","text","retweets"]]
df_tweets = df_tweets.dropna()
l1= df_tweets["text"].values
for i in range(len(l1)):
    l1[i] = (l1[i]+' ')*(int(df_tweets["retweets"].values[i]+1))
df_tweets["text"] =  l1
df_tweets = df_tweets[["date","text"]]
df_tweets.head()

,date,text
0,12/31/2019 23:45,Thinking about transitioning your business? Yo...
1,12/31/2019 23:31,http://RipsawWealth.com #2020WillBeTheYearFor ...
2,12/31/2019 23:00,#wealthmanagement #maxous
3,12/31/2019 22:45,#WealthManagement and private banking trends: ...
4,12/31/2019 22:43,Is Your Money Ready for Retirement? #Retiremen...


In [4]:
def clean_tweets(df=df_tweets, 
                 tweet_col='text', 
                 date_col='date'
                ):
    
    df_copy = df.copy()
    
    # drop rows with empty values
    df_copy.dropna(inplace=True)
    
    # lower the tweets
    df_copy['preprocessed_' + tweet_col] = df_copy[tweet_col].str.lower()
    
    # filter out stop words and URLs
    en_stop_words = stopwords.words('english')
    extended_words = ['&amp;', 'rt','th','co', 're', 've', 'kim', 'daca', '#wealthmanagement', 'wealthmanagement', 'de', 'la', 'en', 'day', 'di', 'el', 'ask', 'un', 'con', 'per', 'del', 'help', 'you', 'via', 'us', 'gt', 'ai', 'cx', 'you', 'time', 'make', 'get', 'one', 'uk']
    en_stop_words.extend(extended_words)
    en_stop_words = set(en_stop_words)
    url_re = '(https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|www\.[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9]+\.[^\s]{2,}|www\.[a-zA-Z0-9]+\.[^\s]{2,})'        
    df_copy['preprocessed_' + tweet_col] = df_copy['preprocessed_' + tweet_col].apply(lambda row: ' '.join([word.strip() for word in row.split() if (not word.strip() in en_stop_words) and (not re.match(url_re, word.strip()))]))
    
    # tokenize the tweets
    tokenizer = RegexpTokenizer('[a-zA-Z]\w+\'?\w*')
    df_copy['tokenized_' + tweet_col] = df_copy['preprocessed_' + tweet_col].apply(lambda row: tokenizer.tokenize(row))
    
    return df_copy
  
df_tweets_clean = clean_tweets(df_tweets)
df_tweets_clean

,date,text,preprocessed_text,tokenized_text
0,12/31/2019 23:45,Thinking about transitioning your business? Yo...,thinking transitioning business? you’ll surpri...,"[thinking, transitioning, business, you, ll, s..."
1,12/31/2019 23:31,http://RipsawWealth.com #2020WillBeTheYearFor ...,#2020willbetheyearfor financial control! #happ...,"[willbetheyearfor, financial, control, happyne..."
2,12/31/2019 23:00,#wealthmanagement #maxous,#maxous,[maxous]
3,12/31/2019 22:45,#WealthManagement and private banking trends: ...,private banking trends: 2020 | @fintechmagazin...,"[private, banking, trends, fintechmagazine, we..."
4,12/31/2019 22:43,Is Your Money Ready for Retirement? #Retiremen...,money ready retirement? #retirementplan #finan...,"[money, ready, retirement, retirementplan, fin..."
...,...,...,...,...
26951,3/19/2019 17:11,.@Praemium said its major technology upgrade w...,.@praemium said major technology upgrade expan...,"[praemium, said, major, technology, upgrade, e..."
26952,3/19/2019 16:59,A Seismic Shift in the Estate-Planning Landsca...,seismic shift estate-planning landscape #estat...,"[seismic, shift, estate, planning, landscape, ..."
26953,3/19/2019 16:58,"Interview: Scott Cooper, @CybiWealth – investi...","interview: scott cooper, @cybiwealth – investi...","[interview, scott, cooper, cybiwealth, investi..."
26954,3/19/2019 16:18,How much are you worth? . . . . #wealth #Wealt...,much worth? . . . . #wealth #wealthy #wealthbu...,"[much, worth, wealth, wealthy, wealthbuilding,..."


In [6]:
# build a dictionary where for each tweet, each word has its own id.
tweets_dictionary = Dictionary(df_tweets_clean.tokenized_text)

# build the corpus i.e. vectors with the number of occurence of each word per tweet
tweets_corpus = [tweets_dictionary.doc2bow(tweet) for tweet in df_tweets_clean.tokenized_text]

In [7]:
import gensim
list_tfidf = []
tfidf = gensim.models.TfidfModel(tweets_corpus)
for doc in tfidf[tweets_corpus]:
    list_tfidf.append([[tweets_dictionary[id], np.around(freq, decimals=2)] for id, freq in doc])

In [8]:
list_1 = []
list_3 = []
for i in range(len(list_tfidf)):
    list_2 = []
    list_4 = []
    for j in range(len(list_tfidf[i])):
        list_2.append(list_tfidf[i][j][0])
        list_4.append(list_tfidf[i][j][1])
    list_1.append(list_2)
    list_3.append(list_4)   

In [9]:
tfidf_df = pd.DataFrame({"Tweets":list_1, "tfidf_score":list_3})
tfidf_df.to_csv("tfidf.csv")

In [10]:
tfidf_df

,Tweets,tfidf_score
0,"[along, business, businessowner, early, find, ...","[0.27, 0.13, 0.26, 0.23, 0.16, 0.21, 0.38, 0.2..."
1,"[control, financial, fintech, happynewyear, ha...","[0.3, 0.13, 0.13, 0.45, 0.48, 0.36, 0.56]"
2,[maxous],[1.0]
3,"[fintech, banking, finserv, fintechmagazine, p...","[0.18, 0.27, 0.28, 0.73, 0.3, 0.36, 0.23]"
4,"[financialplan, money, ready, retirement, reti...","[0.53, 0.43, 0.44, 0.23, 0.54]"
...,...,...
26951,"[market, overall, technology, said, platform, ...","[0.24, 0.23, 0.14, 0.2, 0.16, 0.16, 0.21, 0.37..."
26952,"[estateplanning, planning, landscape, estate, ...","[0.16, 0.13, 0.47, 0.17, 0.48, 0.12, 0.65, 0.1..."
26953,"[fintech, finserv, wealthtech, investing, smal...","[0.12, 0.18, 0.15, 0.1, 0.28, 0.23, 0.26, 0.37..."
26954,"[wealth, wealthbuilder, wealthbuilding, wealth...","[0.05, 0.24, 0.16, 0.15, 0.22, 0.12, 0.23, 0.2..."
